In [23]:
import requests
import pandas as pd
import numpy as np
import urllib.request

In [24]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

In [25]:
def location_splitter(location):
    location=location.split(' ')
    return location[0],location[1],location[2],location[3]

In [26]:
def roadnamegrabber(latitude,longitude):
    tempResponse=requests.get('https://developers.onemap.sg/privateapi/commonsvc/revgeocode?location=%s,%s&token=%s&buffer=100&addressType=all'%(str(latitude),str(longitude),onemapAPItoken))
    name=pd.DataFrame(eval(tempResponse.content)['GeocodeInfo'])['ROAD'].mode()[0]
    return name.replace('S','s').title()

# Onemap

In [27]:
onemapTokenAPIResponse=requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken',json={'email':'leejin@u.nus.edu','password':'Whysohardtochange123!'})#.content
onemapAPItoken=eval(onemapTokenAPIResponse.content)['access_token']

In [28]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])
traffic_image_df['roadname']=np.vectorize(roadnamegrabber)(traffic_image_df['Latitude'],traffic_image_df['Longitude'])
traffic_image_df['Count']=np.random.uniform(low=0, high=20, size=(86,)).astype(int)
traffic_image_df['is_jam']=0

In [29]:
traffic_image_df=traffic_image_df.merge(pd.read_csv('traffic_camera_region.csv',converters={'CameraID':str}),'left','CameraID')

In [30]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])


# NEA API
For some reason this one need to constantly call and keep track of previous results. Sometimes calls does not return all data.
Some names are given as S123, etc, looking to try OneMap reverse geocoding to get road name from lat long.

In [31]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

# Calculations


In [32]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

In [33]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Message']]

In [41]:
final_df=traffic_image_df.merge(weather_df,'outer','key')

final_df['distance_from_id']=(np.vectorize(haversine)(final_df['Latitude'],final_df['Longitude'],final_df['latitude'],final_df['longitude']))
final_df=final_df.sort_values('distance_from_id').groupby('CameraID').head(1)[['CameraID','Latitude','Longitude','Region','rainfall','ImageLink','roadname','Count','is_jam']]
final_df=final_df.sort_values('CameraID').reset_index(drop=True)


In [42]:
final_df

,CameraID,Latitude,Longitude,Region,rainfall,ImageLink,roadname,Count,is_jam
0,1001,1.295313,103.871146,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Tanjong Rhu Road,3,0
1,1002,1.319541,103.878563,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Sims Drive,3,0
2,1003,1.323957,103.872858,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Kallang Sector,13,0
3,1004,1.319536,103.875067,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Geylang Bahru,11,0
4,1005,1.363520,103.905394,East,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Greenwich Drive,16,0
...,...,...,...,...,...,...,...,...,...
81,9702,1.394741,103.817971,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Seletar Expressway,17,0
82,9703,1.422857,103.773005,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Seletar Expressway,15,0
83,9704,1.422143,103.795421,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Mandai Road,12,0
84,9705,1.426277,103.787166,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,Champions Way,17,0


In [43]:
final_df.to_csv('main_df.csv',index=False)
nearest_incidents.to_csv('traffic_incidents.csv',index=False)

# END